In [1]:
from rdkit.Chem import AllChem
from scipy import stats
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from scipy import stats
from rdkit.Chem import rdMolDescriptors as rdmd
from rdkit.Chem.Scaffolds import MurckoScaffold
import pandas as pd
from tqdm import tqdm
import time
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef,confusion_matrix, roc_auc_score, roc_curve
import seaborn as sns
import time
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

In [2]:
import pandas as pd
import numpy as np

In [3]:
assay_data = pd.read_csv("./assay_data_filtered_89_inchi.csv") 
assay_list= assay_data.columns.to_list()[1:-1]
assay_data = assay_data.drop(columns= ["smiles_r"])
assay_data

,InChICode_standardised,178_684,180_686,183_604,184_606,185_665,185_666,186_668,190_628,190_629,...,273_751,274_752,275_753,275_754,276_755,278_757,279_758,280_759,283_762,93_259
0,InChI=1S/C20H29NO5S/c1-13-5-3-4-6-14(2)9-18(22...,1.0,1.0,NaN,1.0,NaN,NaN,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,InChI=1S/C22H26N2O3/c1-25-20-9-5-18(6-10-20)4-...,1.0,1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,InChI=1S/C10H12ClN5O3/c11-10-14-8(12)7-9(15-10...,1.0,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,InChI=1S/C31H39F3N4O5/c1-19-16-38(20(2)18-39)2...,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
4,InChI=1S/C36H40N4O4/c1-25-21-40(26(2)24-41)35(...,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15267,InChI=1S/C29H42O10/c1-15-22(32)23(33)24(34)25(...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
15268,InChI=1S/C9H11N3O4/c10-5-1-2-12-8-7(16-9(12)11...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
15269,InChI=1S/C22H22O8/c1-25-14-5-10(6-15(26-2)20(1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
15270,"InChI=1S/C40H71NO14/c1-15-27-40(11,48)33(44)22...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [4]:
CP_features = pd.read_csv("./CP_filtered_15272.csv") 
CP_features_list= CP_features.columns.to_list()[:-1]
#CP_features = CP_features.to_numpy()
CP_features

,Cells_Correlation_Correlation_RNA_AGP,Cells_Correlation_Costes_AGP_RNA,Cells_Correlation_K_AGP_DNA,Cells_Correlation_K_DNA_AGP,Cells_Correlation_K_DNA_ER,Cells_Correlation_K_DNA_Mito,Cells_Correlation_K_ER_DNA,Cells_Correlation_K_ER_Mito,Cells_Correlation_K_Mito_DNA,Cells_Correlation_K_RNA_DNA,...,Nuclei_Texture_SumVariance_AGP_10_0,Nuclei_Texture_SumVariance_DNA_10_0,Nuclei_Texture_SumVariance_ER_10_0,Nuclei_Texture_SumVariance_Mito_10_0,Nuclei_Texture_SumVariance_RNA_10_0,Nuclei_Texture_Variance_AGP_10_0,Nuclei_Texture_Variance_ER_10_0,Nuclei_Texture_Variance_Mito_10_0,Nuclei_Texture_Variance_RNA_10_0,InChICode_standardised
0,0.213520,0.052182,0.340759,-0.107108,0.030104,0.043808,-0.058823,-0.076555,-0.080047,-0.060985,...,0.127041,0.300797,1.601798,0.649545,0.117986,-0.124655,0.875722,0.060316,-0.108497,InChI=1S/C20H29NO5S/c1-13-5-3-4-6-14(2)9-18(22...
1,0.500980,-0.049488,0.034006,0.009368,0.042325,0.289250,0.072184,0.236915,-0.090521,0.025497,...,0.113111,-0.031291,0.736375,-0.052299,-0.182771,0.020039,0.309519,0.146067,-0.135633,InChI=1S/C22H26N2O3/c1-25-20-9-5-18(6-10-20)4-...
2,0.554610,-0.037230,-0.144126,0.907236,0.762857,0.869041,-0.197808,0.050745,-0.265411,-0.223704,...,0.402068,-0.528586,0.894280,1.096441,0.201130,0.023909,0.209528,0.384470,0.242193,InChI=1S/C10H12ClN5O3/c11-10-14-8(12)7-9(15-10...
3,0.065095,0.052121,-0.021547,0.185901,0.081906,0.034104,-0.030566,-0.068275,-0.000007,-0.060455,...,-0.356829,-0.001533,0.040114,-0.145918,0.025249,-0.111571,0.086286,-0.083360,0.011623,InChI=1S/C31H39F3N4O5/c1-19-16-38(20(2)18-39)2...
4,0.116754,-0.089409,-0.006915,-0.116876,-0.017143,0.037999,0.008466,0.130419,-0.026270,0.015191,...,-0.565546,-0.050469,-0.191591,-0.484524,-0.230474,-0.250742,-0.028181,-0.188926,-0.048785,InChI=1S/C36H40N4O4/c1-25-21-40(26(2)24-41)35(...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15267,0.600421,0.006980,0.341992,-0.482678,-0.421343,0.160259,0.486249,0.404443,0.035852,0.378471,...,0.854714,0.141265,1.257099,0.942200,0.412648,1.143416,1.082932,0.892884,1.190745,InChI=1S/C29H42O10/c1-15-22(32)23(33)24(34)25(...
15268,0.357634,-0.102040,-0.155204,1.278826,0.829811,0.756359,-0.187745,-0.028950,-0.237447,-0.202268,...,-0.235418,-1.117448,-0.116413,0.036485,0.631407,-0.277122,-0.071896,-0.108458,0.260039,InChI=1S/C9H11N3O4/c10-5-1-2-12-8-7(16-9(12)11...
15269,0.385746,-0.010448,0.021540,-0.015133,-0.165300,0.102597,0.144109,0.218774,-0.041078,0.056075,...,0.822444,0.501358,0.956281,1.116960,0.726610,0.275630,0.493553,0.230773,0.175383,InChI=1S/C22H22O8/c1-25-14-5-10(6-15(26-2)20(1...
15270,0.078460,0.015044,-0.009505,-0.009737,-0.060708,-0.000053,-0.001857,0.068244,-0.021163,0.063623,...,-0.252019,0.261459,-0.101648,-0.197162,-0.011630,-0.169541,-0.096224,-0.132107,0.046411,"InChI=1S/C40H71NO14/c1-15-27-40(11,48)33(44)22..."


In [5]:
mfp_features = pd.read_csv("./Mfp_filtered_15272.csv") 
mfp_features_list = mfp_features.columns.to_list()[:-1]
mfp_features['fp'] = [(x) for x in mfp_features.iloc[:, :-1].to_numpy()]
mfp_features = mfp_features[["InChICode_standardised", "fp"]]
mfp_features

,InChICode_standardised,fp
0,InChI=1S/C20H29NO5S/c1-13-5-3-4-6-14(2)9-18(22...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,InChI=1S/C22H26N2O3/c1-25-20-9-5-18(6-10-20)4-...,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,InChI=1S/C10H12ClN5O3/c11-10-14-8(12)7-9(15-10...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
3,InChI=1S/C31H39F3N4O5/c1-19-16-38(20(2)18-39)2...,"[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, ..."
4,InChI=1S/C36H40N4O4/c1-25-21-40(26(2)24-41)35(...,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
...,...,...
15267,InChI=1S/C29H42O10/c1-15-22(32)23(33)24(34)25(...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
15268,InChI=1S/C9H11N3O4/c10-5-1-2-12-8-7(16-9(12)11...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
15269,InChI=1S/C22H22O8/c1-25-14-5-10(6-15(26-2)20(1...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
15270,"InChI=1S/C40H71NO14/c1-15-27-40(11,48)33(44)22...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."


In [6]:
broad_data= pd.merge(assay_data, CP_features)
broad_data= pd.merge(broad_data, mfp_features)
broad_data

,InChICode_standardised,178_684,180_686,183_604,184_606,185_665,185_666,186_668,190_628,190_629,...,Nuclei_Texture_SumVariance_AGP_10_0,Nuclei_Texture_SumVariance_DNA_10_0,Nuclei_Texture_SumVariance_ER_10_0,Nuclei_Texture_SumVariance_Mito_10_0,Nuclei_Texture_SumVariance_RNA_10_0,Nuclei_Texture_Variance_AGP_10_0,Nuclei_Texture_Variance_ER_10_0,Nuclei_Texture_Variance_Mito_10_0,Nuclei_Texture_Variance_RNA_10_0,fp
0,InChI=1S/C20H29NO5S/c1-13-5-3-4-6-14(2)9-18(22...,1.0,1.0,NaN,1.0,NaN,NaN,1.0,1.0,1.0,...,0.127041,0.300797,1.601798,0.649545,0.117986,-0.124655,0.875722,0.060316,-0.108497,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,InChI=1S/C22H26N2O3/c1-25-20-9-5-18(6-10-20)4-...,1.0,1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,...,0.113111,-0.031291,0.736375,-0.052299,-0.182771,0.020039,0.309519,0.146067,-0.135633,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,InChI=1S/C10H12ClN5O3/c11-10-14-8(12)7-9(15-10...,1.0,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,...,0.402068,-0.528586,0.894280,1.096441,0.201130,0.023909,0.209528,0.384470,0.242193,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
3,InChI=1S/C31H39F3N4O5/c1-19-16-38(20(2)18-39)2...,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,-0.356829,-0.001533,0.040114,-0.145918,0.025249,-0.111571,0.086286,-0.083360,0.011623,"[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, ..."
4,InChI=1S/C36H40N4O4/c1-25-21-40(26(2)24-41)35(...,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,-0.565546,-0.050469,-0.191591,-0.484524,-0.230474,-0.250742,-0.028181,-0.188926,-0.048785,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15267,InChI=1S/C29H42O10/c1-15-22(32)23(33)24(34)25(...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.854714,0.141265,1.257099,0.942200,0.412648,1.143416,1.082932,0.892884,1.190745,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
15268,InChI=1S/C9H11N3O4/c10-5-1-2-12-8-7(16-9(12)11...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.235418,-1.117448,-0.116413,0.036485,0.631407,-0.277122,-0.071896,-0.108458,0.260039,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
15269,InChI=1S/C22H22O8/c1-25-14-5-10(6-15(26-2)20(1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.822444,0.501358,0.956281,1.116960,0.726610,0.275630,0.493553,0.230773,0.175383,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
15270,"InChI=1S/C40H71NO14/c1-15-27-40(11,48)33(44)22...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.252019,0.261459,-0.101648,-0.197162,-0.011630,-0.169541,-0.096224,-0.132107,0.046411,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."


In [7]:
def TanimotoSimilarity(ex1, ex2, attrs):
  """
    >>> v1 = [0,1,0,1]
    >>> v2 = [1,0,1,0]
    >>> TanimotoDist(v1,v2,range(4))
    1.0
    """
  inter = 0.0
  unin = 0.0
  for i in attrs:
    if (ex1[i] or ex2[i]):
      unin += 1
      if (ex1[i] and ex2[i]):
        inter += 1
  if (unin != 0.0):
    return (inter / unin)
  else:
    return 0

In [8]:
def BulkTanimoto(siv1, sivs):
    res = []
    for i in range(len(sivs)): 
        siv2 = sivs[i]
        simVal = TanimotoSimilarity(siv1, siv2, range(len(siv1)))
        res.append(simVal)
    return res

In [9]:
[1,2, 3,4][::-1]

[4, 3, 2, 1]

In [70]:
#Find similarity between each test compounds and train set compounds
#mean of the 5 highest tanimoto similarity of MFP keys, all, actives, inactives
#perc of compound with MFP TS>0.55, all, actives, inactives
neighbours_limit = 5
def findsimscore(assay, train, test): 
    
    ts_nearest_list =[]
    ts_nearest_actives_list=[]
    ts_nearest_inactives_list=[]
    perc_list=[]
    perc_actives_list=[]
    perc_inactives_list=[]
    
    for index, row in test.iterrows():

        fps_test = row["fp"]

        temp_train = pd.DataFrame()
        temp_train= train

        fps_temp_train=[(x) for x in temp_train["fp"]]
        ts_nearest = []
        ts_nearest = BulkTanimoto(fps_test ,fps_temp_train)

        #Add all values with Activity
        df_temp = pd.DataFrame([temp_train[[assay]].values.flatten(), ts_nearest]).T
        df_temp.columns = ["Activity", "TS_similarity"]
        
        #Calculate overall percetages
        count = len([i for i in df_temp["TS_similarity"].values if i > 0.55])
        total= len(df_temp)
        perc= float(count/total)  
        perc_list.append(perc)
       
        ts_nearest_ovarall = df_temp.sort_values(by="TS_similarity", ascending=False)["TS_similarity"][:neighbours_limit].mean()
        #print(ts_nearest_ovarall)
        ts_nearest_list.append(ts_nearest_ovarall)
        
        #Calculate percetages active compound
        count = len([i for i in df_temp[df_temp["Activity"]==1]["TS_similarity"].values if i > 0.55])
        total= (len(df_temp[df_temp["Activity"]==1]))
        perc_actives= float(count/total)
        perc_actives_list.append(perc_actives)
        
        ts_nearest_actives = df_temp[df_temp["Activity"]==1].sort_values(by="TS_similarity", ascending=False)["TS_similarity"][:neighbours_limit].mean()
        #print(ts_nearest_actives)
        ts_nearest_actives_list.append(ts_nearest_actives)
        
        #Calculate percetages inactive compound
        count = len([i for i in df_temp[df_temp["Activity"]==0]["TS_similarity"].values if i > 0.55])
        total= (len(df_temp[df_temp["Activity"]==0]))
        perc_inactives= float(count/total)
        perc_inactives_list.append(perc_inactives)
        
        ts_nearest_inactives = df_temp[df_temp["Activity"]==0].sort_values(by="TS_similarity", ascending=False)["TS_similarity"][:neighbours_limit].mean()
        #print(ts_nearest_inactives)
        ts_nearest_inactives_list.append(ts_nearest_inactives)

    ts_nearest_list=np.array(ts_nearest_list)
    ts_nearest_list= ts_nearest_list.reshape(len(test))
    
    ts_nearest_actives_list=np.array(ts_nearest_actives_list)
    ts_nearest_actives_list= ts_nearest_actives_list.reshape(len(test))
    
    ts_nearest_inactives_list=np.array(ts_nearest_inactives_list)
    ts_nearest_inactives_list= ts_nearest_inactives_list.reshape(len(test))

    perc_list=np.array(perc_list)
    perc_list= perc_list.reshape(len(test))
    
    perc_actives_list=np.array(perc_actives_list)
    perc_actives_list= perc_actives_list.reshape(len(test))
    
    perc_inactives_list=np.array(perc_inactives_list)
    perc_inactives_list= perc_inactives_list.reshape(len(test))

    df_ts_nearest = pd.DataFrame({"ts_nearest": ts_nearest_list, "perc": perc_list,
                              "ts_nearest_actives": ts_nearest_actives_list, "perc_actives": perc_actives_list,
                              "ts_nearest_inactives": ts_nearest_inactives_list, "perc_inactives": perc_inactives_list})
    return df_ts_nearest

In [88]:
#Find similarity between each test compounds and train set compounds
#mean of the 5 highest PC keys, all, actives, inactives
#perc of compound with PC TS>0.55, all, actives, inactives

neighbours_limit = 5
def findpearsonscore(assay, train, test): 
    pc_highest_list =[]
    pc_highest_actives_list=[]
    pc_highest_inactives_list=[]
    perc_list=[]
    perc_actives_list=[]
    perc_inactives_list=[]
    
    for index, row in test.iterrows():

        fps_test = row[CP_features_list].to_numpy()

        temp_train = pd.DataFrame()
        temp_train= train

        fps_temp_train=[(x) for x in temp_train[CP_features_list].to_numpy()]
        pearson_s_list = []
        
        for n in range(len(fps_temp_train)):
            
            pearson= stats.pearsonr(fps_test, fps_temp_train[n])
            pearson_s_list.append(pearson[0])
            #mean of 15 positive correlations
            
        #Add all values with Activity
        df_temp = pd.DataFrame([temp_train[[assay]].values.flatten(), pearson_s_list]).T
        df_temp.columns = ["Activity", "PC_corr"]
        
        #Calculate overall percetages
        count = len([i for i in df_temp["PC_corr"].values if i > 0.55])
        total= len(df_temp)
        perc= float(count/total)  
        perc_list.append(perc)
       
        pc_highest_ovarall = df_temp.sort_values(by="PC_corr", ascending=False)["PC_corr"][:neighbours_limit].mean()
        #print(pc_highest_ovarall)
        pc_highest_list.append(pc_highest_ovarall)
        
        #Calculate percetages active compound
        count = len([i for i in df_temp[df_temp["Activity"]==1]["PC_corr"].values if i > 0.50])
        total= (len(df_temp[df_temp["Activity"]==1]))
        perc_actives= float(count/total)
        perc_actives_list.append(perc_actives)
        
        pc_highest_actives = df_temp[df_temp["Activity"]==1].sort_values(by="PC_corr", ascending=False)["PC_corr"][:neighbours_limit].mean()
        #print(pc_highest_actives)
        pc_highest_actives_list.append(pc_highest_actives)
        
        #Calculate percetages inactive compound
        count = len([i for i in df_temp[df_temp["Activity"]==0]["PC_corr"].values if i > 0.50])
        total= (len(df_temp[df_temp["Activity"]==0]))
        perc_inactives= float(count/total)
        perc_inactives_list.append(perc_inactives)
        
        pc_highest_inactives = df_temp[df_temp["Activity"]==0].sort_values(by="PC_corr", ascending=False)["PC_corr"][:neighbours_limit].mean()
        #print(ts_nearest_inactives)
        pc_highest_inactives_list.append(pc_highest_inactives)

    pc_highest_list=np.array(pc_highest_list)
    pc_highest_list= pc_highest_list.reshape(len(test))
    
    pc_highest_actives_list=np.array(pc_highest_actives_list)
    pc_highest_actives_list= pc_highest_actives_list.reshape(len(test))
    
    pc_highest_inactives_list=np.array(pc_highest_inactives_list)
    pc_highest_inactives_list= pc_highest_inactives_list.reshape(len(test))

    perc_list=np.array(perc_list)
    perc_list= perc_list.reshape(len(test))
    
    perc_actives_list=np.array(perc_actives_list)
    perc_actives_list= perc_actives_list.reshape(len(test))
    
    perc_inactives_list=np.array(perc_inactives_list)
    perc_inactives_list= perc_inactives_list.reshape(len(test))

    df_pc_highest = pd.DataFrame({"pc_highest": pc_highest_list, "perc": perc_list,
                              "pc_highest_actives": pc_highest_actives_list, "perc_actives": perc_actives_list,
                              "pc_highest_inactives": pc_highest_inactives_list, "perc_inactives": perc_inactives_list})
    return df_pc_highest

In [89]:
def plot_hist_ts(df):
    
    #f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw= {"height_ratios": (0.2, 1)})
    
    mean=df['ts_nearest'].mean()
    median=df['ts_nearest'].median()
    mode=df['ts_nearest'].mode().tolist()[0]
    
    
    #sns.boxplot(x=df["td_nearest"], ax=ax_box)
    #ax_box.axvline(mean, color='r', linestyle='--',  label='Mean')
    #ax_box.axvline(median, color='g', linestyle='-',  label='Median')
    #ax_box.axvline(mode, color='b', linestyle='-',  label='Mode')

    #sns.histplot(x=df["td_nearest"],  kde=True, ax=ax_hist)
    #ax_hist.axvline(mean, color='r', linestyle='--', label='Mean')
    #ax_hist.axvline(median, color='g', linestyle='-', label='Median')
    #ax_hist.axvline(mode, color='b', linestyle='-', label='Mode')

    #plt.legend()
    #ax_box.set(xlabel='')
    #plt.show()
    
    return mean, median, mode

def plot_hist_pc(df):
    
    #f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw= {"height_ratios": (0.2, 1)})
    
    mean=df['pc_highest'].mean()
    median=df['pc_highest'].median()
    mode=df['pc_highest'].mode().tolist()[0]
    
    
    #sns.boxplot(x=df["pc_highest"], ax=ax_box)
    #ax_box.axvline(mean, color='r', linestyle='--',  label='Mean')
    #ax_box.axvline(median, color='g', linestyle='-',  label='Median')
    #ax_box.axvline(mode, color='b', linestyle='-',  label='Mode')

    #sns.histplot(x=df["pc_highest"],  kde=True, ax=ax_hist)
    #ax_hist.axvline(mean, color='r', linestyle='--', label='Mean')
    #ax_hist.axvline(median, color='g', linestyle='-', label='Median')
    #ax_hist.axvline(mode, color='b', linestyle='-', label='Mode')

    #plt.legend()
    #ax_box.set(xlabel='')
    #plt.show()
    
    return mean, median, mode

In [93]:
#Nested Cross validation
#stat_list_batch: contains assay number, mean, median, mode of tanimoto distance in that cycle, mcc and auc of that cycle
#detail_list_batch contains assay number, each inchi and its proba, pred, true values and ts
#pred_list_batch contains proba, pred and true values

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.under_sampling import RandomUnderSampler 

from collections import Counter
from numpy import argmax

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from scipy.stats import ks_2samp 
from scipy.stats import randint
from sklearn.model_selection import cross_val_predict

def my_function_nested_rhs(assay, cycle):   
        
    detail_list_batch_train = []

    X_fp = df_assay_train.loc[:, "fp"].to_numpy()
    X_CP = df_assay_train[CP_features_list].to_numpy()
    
    #print(X)
    y = df_assay_train[assay].to_numpy()
    #print(y)
    
    
    #Split
    outercv = StratifiedKFold(n_splits=5,shuffle=True, random_state=cycle)
   
    for i, (train_index, test_index) in enumerate(outercv.split(X_fp, y)):
        
        train= df_assay_train.iloc[train_index]
        test= df_assay_train.iloc[test_index]
        #print(train)
        #print(test)

        #df_ts_perc = findsimsperc(assay, train, test) 
        #print
        
        df_ts_highest = findsimscore(assay, train, test)
        df_pc_highest = findpearsonscore(assay, train, test)    
        
        mean_ts, median_score, mode_score= plot_hist_ts(df_ts_highest)
        mean_pc, median_score, mode_score= plot_hist_pc(df_pc_highest)

        #combine results
        for InChICode_standardised, ts, ts_perc, ts_actives, ts_perc_actives, ts_inactives, ts_perc_inactives, pc, pc_perc, pc_actives, pc_perc_actives, pc_inactives, pc_perc_inactives  in zip(test.InChICode_standardised, 
                                                df_ts_highest.ts_nearest, df_ts_highest.perc, df_ts_highest.ts_nearest_actives, df_ts_highest.perc_actives,  df_ts_highest.ts_nearest_inactives, df_ts_highest.perc_inactives,   
                                                  df_pc_highest.pc_highest, df_pc_highest.perc, df_pc_highest.pc_highest_actives, df_pc_highest.perc_actives, df_pc_highest.pc_highest_inactives, df_pc_highest.perc_inactives):
            detail_list_batch_train.append([assay, InChICode_standardised, ts, ts_perc, ts_actives, ts_perc_actives, 
                                            ts_inactives, ts_perc_inactives, pc, pc_perc, pc_actives, 
                                            pc_perc_actives, pc_inactives, pc_perc_inactives, "Training"])
    
    return detail_list_batch_train

In [94]:
#Predict External Test Set
#detail_list_batch contains assay number, each inchi and its proba, pred, true values and ts

def my_function_heldout_rhs(assay, cycle):   
        
        detail_list_batch_test = []

        train= df_assay_train
        test= df_assay_test

        df_ts_highest = findsimscore(assay, train, test)
        df_pc_highest = findpearsonscore(assay, train, test)    
        
        mean_ts, median_score, mode_score= plot_hist_ts(df_ts_highest)
        mean_pc, median_score, mode_score= plot_hist_pc(df_pc_highest)
        
        #combine results
        for InChICode_standardised, ts, ts_perc, ts_actives, ts_perc_actives, ts_inactives, ts_perc_inactives, pc, pc_perc, pc_actives, pc_perc_actives, pc_inactives, pc_perc_inactives  in zip(test.InChICode_standardised, 
                                                df_ts_highest.ts_nearest, df_ts_highest.perc, df_ts_highest.ts_nearest_actives, df_ts_highest.perc_actives,  df_ts_highest.ts_nearest_inactives, df_ts_highest.perc_inactives,   
                                                  df_pc_highest.pc_highest, df_pc_highest.perc, df_pc_highest.pc_highest_actives, df_pc_highest.perc_actives, df_pc_highest.pc_highest_inactives, df_pc_highest.perc_inactives):
            detail_list_batch_test.append([assay, InChICode_standardised, ts, ts_perc, ts_actives, ts_perc_actives, 
                                           ts_inactives, ts_perc_inactives, pc, pc_perc, pc_actives, pc_perc_actives, 
                                           pc_inactives, pc_perc_inactives, "HeldOut"])
    
        return detail_list_batch_test

In [100]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split

master_detail_list=[]
for assay in tqdm(assay_list[:]):
    
    print("Assay ", assay )
    df_assay= broad_data[pd.notnull(broad_data[assay])]

    print("Total Compounds ", len(df_assay) )

    df_assay_inactive = df_assay[df_assay[assay] == 0]
    inactives_n=len(df_assay_inactive)
    print("Inactive Compounds ", inactives_n)

    df_assay_active = df_assay[df_assay[assay] == 1]
    actives_n=len(df_assay_active)
    print("Active Compounds ", actives_n)

    df_assay_train, df_assay_test= train_test_split(df_assay, stratify=df_assay[[assay]], test_size=0.20, random_state=42)
    
    print("Perform nested cross validation on training set")
    detail_list_train =  my_function_nested_rhs(assay, 42)
   
    
    print("Predictions on held out set")
    detail_list_test = my_function_heldout_rhs(assay, 42)

    
    master_detail_list.append(detail_list_train) 
    master_detail_list.append(detail_list_test) 
     

  0%|                                                    | 0/89 [00:00<?, ?it/s]

Assay  178_684
Total Compounds  272
Inactive Compounds  204
Active Compounds  68
Perform nested cross validation on training set
Predictions on held out set


  1%|▍                                           | 1/89 [00:08<12:00,  8.19s/it]

Assay  180_686
Total Compounds  204
Inactive Compounds  153
Active Compounds  51
Perform nested cross validation on training set
Predictions on held out set


  2%|▉                                           | 2/89 [00:13<09:05,  6.27s/it]

...

 98%|████████████████████████████████████████ | 87/89 [1:59:03<01:46, 53.36s/it]

Assay  283_762
Total Compounds  428
Inactive Compounds  321
Active Compounds  107
Perform nested cross validation on training set
Predictions on held out set


 99%|████████████████████████████████████████▌| 88/89 [1:59:21<00:42, 42.77s/it]

Assay  93_259
Total Compounds  127
Inactive Compounds  93
Active Compounds  34
Perform nested cross validation on training set
Predictions on held out set


100%|█████████████████████████████████████████| 89/89 [1:59:23<00:00, 80.49s/it]


In [101]:
flatList_master_detail_list = [ item for elem in master_detail_list for item in elem]
list_of_lists_df = pd.DataFrame(flatList_master_detail_list,columns=["assay","InChICode_standardised",
                                                                    "ts", "ts_perc", "ts_actives", "ts_perc_actives", 
                                                                     "ts_inactives", "ts_perc_inactives", "pc", "pc_perc", "pc_actives", "pc_perc_actives", 
                                                                     "pc_inactives", "pc_perc_inactives", 
                                                                     "Data"])
list_of_lists_df

,assay,InChICode_standardised,ts,ts_perc,ts_actives,ts_perc_actives,ts_inactives,ts_perc_inactives,pc,pc_perc,pc_actives,pc_perc_actives,pc_inactives,pc_perc_inactives,Data
0,178_684,InChI=1S/C46H56N4O10/c1-7-42(55)22-28-23-45(40...,0.290269,0.000000,0.284818,0.000000,0.271792,0.000000,0.859367,0.063584,0.859367,0.209302,0.512414,0.015385,Training
1,178_684,InChI=1S/C30H42ClN3O5S/c1-21-17-34(22(2)20-35)...,0.787685,0.213873,0.660846,0.093023,0.745801,0.253846,0.754709,0.231214,0.729592,0.465116,0.723072,0.246154,Training
2,178_684,InChI=1S/C31H44N4O6/c1-22-18-35(23(2)21-36)31(...,0.772941,0.132948,0.553399,0.023256,0.772941,0.169231,0.557030,0.017341,0.436839,0.046512,0.499498,0.015385,Training
3,178_684,InChI=1S/C35H60N4O5/c1-26-22-39(27(2)25-40)35(...,0.812935,0.208092,0.574854,0.069767,0.812935,0.253846,0.717875,0.104046,0.500185,0.023256,0.716548,0.146154,Training
4,178_684,InChI=1S/C29H39N5O5/c1-19-16-34(20(2)18-35)28(...,0.770504,0.346821,0.670484,0.255814,0.762533,0.376923,0.478680,0.005780,0.370233,0.000000,0.478680,0.007692,Training
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58482,93_259,InChI=1S/C12H11N3O2S/c1-8-7-9(2)15(13-8)12-10-...,0.324773,0.000000,0.279837,0.000000,0.305871,0.000000,0.681960,0.089109,0.509096,0.111111,0.678402,0.121622,HeldOut
58483,93_259,InChI=1S/C17H16N2O4S/c1-22-14-7-8-16(23-2)17(1...,0.316906,0.000000,0.277834,0.000000,0.308875,0.000000,0.504201,0.009901,0.373719,0.037037,0.483532,0.027027,HeldOut
58484,93_259,InChI=1S/C24H21N3O2S/c28-23-19-8-4-5-9-21(19)3...,0.276304,0.000000,0.263956,0.000000,0.261232,0.000000,0.425408,0.000000,0.353781,0.000000,0.423772,0.000000,HeldOut
58485,93_259,InChI=1S/C18H22N2O4S/c1-14(2)15-5-7-16(8-6-15)...,0.343005,0.000000,0.328585,0.000000,0.323085,0.000000,0.628148,0.069307,0.441401,0.074074,0.616700,0.094595,HeldOut


In [102]:
#list_of_lists_df.to_csv("Similarity_def_mean_train_test_no_Classv2.csv", index=False)